In [1]:
# 連結Google drive檔案
from google.colab import drive
import pandas as pd
drive.mount('/content/drive', force_remount=True)

file_path =("/content/EN_generated.xlsx")

Mounted at /content/drive


In [2]:
!pip install openpyxl

In [3]:
df = pd.read_excel(file_path)

# Calculate the total number of news content
total_news_content = len(df)

# Calculate unique news titles


# Impact Duration Distribution based on the exact values of 0, 1, and 2
impact_duration_distribution = df['Impact_Duration'].value_counts().sort_index()

# Impact Level Distribution


# Output the calculated values
print(f"Total News Content: {total_news_content}")

print("Impact Duration Distribution:")
print(impact_duration_distribution)

Total News Content: 7499
Impact Duration Distribution:
Impact_Duration
0    2497
1    2498
2    2504
Name: count, dtype: int64


In [4]:
!pip install transformers
!pip install sklearn

!pip install scikit-learn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import get_linear_schedule_with_warmup, AdamW
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [6]:
# Encode the 'impact_duration' labels to numeric values
label_encoder = LabelEncoder()
df['impact_duration_encoded'] = label_encoder.fit_transform(df['Impact_Duration'])
# Check for NaN values in 'news_content'
if df['news_content'].isnull().any():
    # Handle NaN values, e.g., by replacing them with a placeholder string
    df['news_content'].fillna('No content', inplace=True)

# Split the data into training and validation sets (80% training, 20% validation)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')


def tokenize_data(df):
    # Ensure all entries are strings and handle missing values if necessary
    texts = df['news_content'].astype(str).tolist()  # Convert to string to avoid issues
    return tokenizer(texts, truncation=True, padding=True, max_length=512)


train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)

train_labels = train_df['impact_duration_encoded'].tolist()
val_labels = val_df['impact_duration_encoded'].tolist()

# Create a custom dataset for PyTorch
class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

# Load the BERT model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=len(label_encoder.classes_))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [8]:
!pip install transformers -U
!pip install accelerate -U
#!pip install accelerate ==0.27.2
!pip install scikit-learn

In [9]:
import accelerate
print(accelerate.__version__)

0.29.3


In [10]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
)

# Compute metrics function for evaluation
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    # Calculate weighted precision, recall, and F1
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted')

    # Calculate macro F1
    _, _, f1_macro, _ = precision_recall_fscore_support(labels, predictions, average='macro')

    # Calculate micro F1
    _, _, f1_micro, _ = precision_recall_fscore_support(labels, predictions, average='micro')

    # Calculate accuracy
    acc = accuracy_score(labels, predictions)

    # Return all metrics
    return {
        'accuracy': acc,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'precision_weighted': precision_weighted,
        'recall_weighted': recall_weighted
    }

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()

print(evaluation_results)


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,F1 Micro,Precision Weighted,Recall Weighted
1,0.749300,0.327256,0.916000,0.916524,0.916209,0.916000,0.918753,0.916000
2,0.204100,0.193880,0.962667,0.962498,0.962273,0.962667,0.963060,0.962667
3,0.074600,0.112920,0.981333,0.981344,0.981179,0.981333,0.981371,0.981333


{'eval_loss': 0.1129203587770462, 'eval_accuracy': 0.9813333333333333, 'eval_f1_weighted': 0.9813442737672544, 'eval_f1_macro': 0.9811787189848458, 'eval_f1_micro': 0.9813333333333333, 'eval_precision_weighted': 0.9813709338680413, 'eval_recall_weighted': 0.9813333333333333, 'eval_runtime': 5.2273, 'eval_samples_per_second': 286.954, 'eval_steps_per_second': 17.982, 'epoch': 3.0}


In [11]:

# Generate predictions for the training dataset
train_predictions = trainer.predict(train_dataset)

# Generate predictions for the validation dataset
val_predictions = trainer.predict(val_dataset)

# The predictions are in logits, so you need to apply softmax to convert to probabilities and then take the argmax to get the predicted labels
train_pred_labels = np.argmax(train_predictions.predictions, axis=1)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)

# The true labels are already provided as part of the datasets
train_true_labels = train_predictions.label_ids
val_true_labels = val_predictions.label_ids

# Calculate accuracy for the training dataset
train_accuracy = accuracy_score(train_true_labels, train_pred_labels)

# Calculate accuracy for the validation dataset
val_accuracy = accuracy_score(val_true_labels, val_pred_labels)

# To print classification reports, you'll need to import it
from sklearn.metrics import classification_report

# Convert the label encoder's classes to strings for use as target names
target_names = [str(label) for label in label_encoder.inverse_transform(range(len(label_encoder.classes_)))]

# Then use these target names in your classification report
print("Training Dataset Metrics:")
print(f"Accuracy: {train_accuracy}")
print(classification_report(train_true_labels, train_pred_labels, target_names=target_names))

print("Validation Dataset Metrics:")
print(f"Accuracy: {val_accuracy}")
print(classification_report(val_true_labels, val_pred_labels, target_names=target_names))



Training Dataset Metrics:
Accuracy: 0.9971661943657276
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1982
           1       1.00      1.00      1.00      2005
           2       1.00      0.99      1.00      2012

    accuracy                           1.00      5999
   macro avg       1.00      1.00      1.00      5999
weighted avg       1.00      1.00      1.00      5999

Validation Dataset Metrics:
Accuracy: 0.9813333333333333
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       515
           1       0.98      0.97      0.98       493
           2       0.97      0.98      0.98       492

    accuracy                           0.98      1500
   macro avg       0.98      0.98      0.98      1500
weighted avg       0.98      0.98      0.98      1500



Adam optimizer

In [ ]:
from transformers import get_linear_schedule_with_warmup, AdamW


# Example of setting up an optimizer with weight decay
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# Example of setting up a learning rate scheduler
num_training_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Update training_args to include gradient clipping
training_args.gradient_accumulation_steps = 1
training_args.max_grad_norm = 1.0

# Implementing a Trainer with custom optimizer and scheduler
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)  # Pass custom optimizer and scheduler
)

# Fine-tuning with the Trainer
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,F1 Micro,Precision Weighted,Recall Weighted
1,0.171600,0.133593,0.978667,0.978670,0.978668,0.978667,0.978783,0.978667
2,0.050400,0.120807,0.984667,0.984653,0.984581,0.984667,0.984690,0.984667
3,0.010100,0.110501,0.987333,0.987331,0.987300,0.987333,0.987340,0.987333


TrainOutput(global_step=2250, training_loss=0.07799337196350098, metrics={'train_runtime': 463.8877, 'train_samples_per_second': 38.796, 'train_steps_per_second': 4.85, 'total_flos': 2497105641190860.0, 'train_loss': 0.07799337196350098, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy for the training dataset
train_accuracy = accuracy_score(train_true_labels, train_pred_labels)

# Calculate accuracy for the validation dataset
val_accuracy = accuracy_score(val_true_labels, val_pred_labels)

# Print accuracy along with the classification report
print("Training Dataset Metrics:")
print(f"Accuracy: {train_accuracy}")
print(classification_report(train_true_labels, train_pred_labels))

print("Validation Dataset Metrics:")
print(f"Accuracy: {val_accuracy}")
# print(classification_report(val_true_labels, val_pred_labels))

Training Dataset Metrics:
Accuracy: 0.9971661943657276
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1982
           1       1.00      1.00      1.00      2005
           2       1.00      0.99      1.00      2012

    accuracy                           1.00      5999
   macro avg       1.00      1.00      1.00      5999
weighted avg       1.00      1.00      1.00      5999

Validation Dataset Metrics:
Accuracy: 0.9813333333333333


Trainer - learning rate adjustment

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Assuming you have the `Trainer` and `TrainingArguments` set up
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Number of training steps
num_train_steps = len(trainer.get_train_dataloader()) * training_args.num_train_epochs

# Setup learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=num_train_steps)

trainer.create_optimizer_and_scheduler(num_train_steps)

trainer.optimizer = optimizer
trainer.lr_scheduler = scheduler

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,F1 Micro,Precision Weighted,Recall Weighted
1,0.060200,0.299787,0.948667,0.948614,0.948567,0.948667,0.950012,0.948667
2,0.071600,0.130056,0.978667,0.978667,0.978663,0.978667,0.978933,0.978667
3,0.020900,0.119384,0.983333,0.983328,0.983251,0.983333,0.983423,0.983333


TrainOutput(global_step=2250, training_loss=0.08801228099399143, metrics={'train_runtime': 436.5195, 'train_samples_per_second': 41.228, 'train_steps_per_second': 5.154, 'total_flos': 2497105641190860.0, 'train_loss': 0.08801228099399143, 'epoch': 3.0})

In [ ]:
# Calculate accuracy for the training dataset
train_accuracy = accuracy_score(train_true_labels, train_pred_labels)

# Generate and print classification report for the training dataset
print("Training Dataset Metrics:")
print(f"Accuracy: {train_accuracy}")
print(classification_report(train_true_labels, train_pred_labels))
# Calculate accuracy for the validation dataset
val_accuracy = accuracy_score(val_true_labels, val_pred_labels)

# Generate and print classification report for the validation dataset
print("Validation Dataset Metrics:")
print(f"Accuracy: {val_accuracy}")
print(classification_report(val_true_labels, val_pred_labels))


Training Dataset Metrics:
Accuracy: 0.9971661943657276
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1982
           1       1.00      1.00      1.00      2005
           2       1.00      0.99      1.00      2012

    accuracy                           1.00      5999
   macro avg       1.00      1.00      1.00      5999
weighted avg       1.00      1.00      1.00      5999

Validation Dataset Metrics:
Accuracy: 0.9813333333333333
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       515
           1       0.98      0.97      0.98       493
           2       0.97      0.98      0.98       492

    accuracy                           0.98      1500
   macro avg       0.98      0.98      0.98      1500
weighted avg       0.98      0.98      0.98      1500

